In [ ]:
import uproot
import awkward as ak
import vector
import numpy as np
import scipy.constants as const
import pyarrow.feather as feather
import pandas as pd

m_e = const.electron_mass / 1000
m_mu = 0.105658

def import_raw_data():
    tree_plus = uproot.open('../Delphes/output_WWjj_e+mu_smeft_cWtil_NP1_tot.root:Delphes')
    plus_events = tree_plus.arrays(['Event.Weight', 'Electron_size', 'Electron.Eta', 'Electron.PT',
                           'Electron.Phi', 'Muon_size', 'Muon.PT', 'Muon.Eta', 'Muon.Phi',
                           'Jet_size', 'Jet.PT', 'Jet.Eta', 'Jet.Phi','Jet.Mass','Jet.BTag',])
    
    return plus_events

def di_invariant_mass_calc(particles):
    diparticle_mass = (particles[:, 0] + particles[:, 1]).mass
    return diparticle_mass

def cross_invariant_mass_calc(particle_1, particle_2):
    diparticle_mass = (particle_1[:, 0] + particle_2[:, 0]).mass
    return diparticle_mass

def delta_eta(particles):
    return np.abs(particles[:, 0].eta - particles[:, 1].eta)

def electron_muon_cuts(events):
    weights = events['Event.Weight']

    single_electron_muon_mask = (events['Electron_size'] == 1) & (events['Muon_size'] == 1)
    unfiltered_events = events[single_electron_muon_mask]

    electron_pt = unfiltered_events['Electron.PT']
    electron_eta = unfiltered_events['Electron.Eta']
    muon_pt = unfiltered_events['Muon.PT']
    muon_eta = unfiltered_events['Muon.Eta']

    electron_pt_mask = electron_pt > 20
    electron_eta_mask = np.abs(electron_eta) < 2.47
    exclusion_mask = (np.abs(electron_eta) < 1.37) | (np.abs(electron_eta) > 1.52)
    total_electron_mask = electron_pt_mask & electron_eta_mask & exclusion_mask
    muon_pt_mask = muon_pt > 20
    muon_eta_mask = np.abs(muon_eta) < 2.5
    total_muon_mask = muon_pt_mask & muon_eta_mask

    filtered_mask = (ak.sum(total_electron_mask, axis=1) == 1) & (ak.sum(total_muon_mask, axis=1) == 1)
 
    highest_pt_lepton_mask = (electron_pt >= muon_pt)
    highest_pt_cut = 27
    highest_pt_mask = ((highest_pt_lepton_mask & (electron_pt > highest_pt_cut)) | 
                       (~highest_pt_lepton_mask & (muon_pt > highest_pt_cut)))
    
    total_electron_mask = total_electron_mask & highest_pt_mask
    filtered_mask = (ak.sum(total_electron_mask, axis=1) == 1) & (ak.sum(total_muon_mask, axis=1) == 1)
    
    filtered_events = unfiltered_events[filtered_mask]
    filtered_weights = weights[filtered_mask]

    filtered_electrons = electron_vector(filtered_events)
    filtered_muons = muon_vector(filtered_events)

    filtered_electron_muon_mass = cross_invariant_mass_calc(filtered_electrons, filtered_muons)
    electron_muon_invariant_mass_mask = (filtered_electron_muon_mass > 40) & (filtered_electron_muon_mass < 400)
    filtered_electron_muon_mass = filtered_electron_muon_mass[electron_muon_invariant_mass_mask]

    filtered_events = filtered_events[electron_muon_invariant_mass_mask]
    filtered_weights = filtered_weights[electron_muon_invariant_mass_mask]

    return filtered_events, filtered_weights

def jet_cuts(filtered_events, events, filtered_weights, k_factor):
    jet_b_tag = filtered_events['Jet.BTag'] == 0
    jet_pt = filtered_events['Jet.PT']
    jet_eta = filtered_events['Jet.Eta']

    forward_jet_mask = ((np.abs(filtered_events['Jet.Eta']) > 2.5) & (np.abs(filtered_events['Jet.Eta']) < 4.5) & (filtered_events['Jet.PT'] > 30)) & jet_b_tag
    central_jet_mask = ((np.abs(filtered_events['Jet.Eta']) < 2.5) & (filtered_events['Jet.PT'] > 20)) & jet_b_tag
    central_jet_region_mask = (abs(filtered_events['Jet.Eta']) < 2.5)

    total_forward_central_mask = forward_jet_mask | central_jet_mask
    filtered_central_jet_mask = (ak.sum(central_jet_region_mask, axis=1) >= 2) & ak.all(total_forward_central_mask, axis=1)

    jet_eta = filtered_events['Jet.Eta'][filtered_central_jet_mask]
    jet_pt = filtered_events['Jet.PT'][filtered_central_jet_mask]
    jet_phi = filtered_events['Jet.Phi'][filtered_central_jet_mask]
    jet_mass = filtered_events['Jet.Mass'][filtered_central_jet_mask]

    jet_pt_masked = jet_pt.mask[jet_eta < 2.5]
    jet_eta_masked = jet_eta.mask[jet_eta < 2.5]
    jet_phi_masked = jet_phi.mask[jet_eta < 2.5]
    jet_mass_masked = jet_mass.mask[jet_eta < 2.5]
    jet_pt_masked = ak.drop_none(jet_pt_masked, highlevel=True)
    jet_eta_masked = ak.drop_none(jet_eta_masked, highlevel=True)
    jet_phi_masked = ak.drop_none(jet_phi_masked, highlevel=True)
    jet_mass_masked = ak.drop_none(jet_mass_masked, highlevel=True)

    jets_central = vector.zip({
        'pt':jet_pt_masked,
        'eta':jet_eta_masked,
        'phi':jet_phi_masked,
        'mass':jet_mass_masked
    })

    central_jet_mass = di_invariant_mass_calc(jets_central)
    central_jet_eta = delta_eta(jets_central)

    jets_central_mass_cut_mask = (central_jet_mass < 160)
    jets_central_delta_eta_cut_mask = (central_jet_eta < 1.5)

    central_jet_cut_mask = jets_central_mass_cut_mask & jets_central_delta_eta_cut_mask

    filtered_events = filtered_events[filtered_central_jet_mask]
    filtered_events = filtered_events[central_jet_cut_mask]
    filtered_weights = filtered_weights[filtered_central_jet_mask]
    filtered_weights = filtered_weights[central_jet_cut_mask]

    weight_normalisation = NP_normalisation(events, k_factor)
    filtered_weights = (weight_normalisation * np.ones_like(filtered_weights)) * filtered_weights

    print("Number of events passing the selection cuts is:", len(filtered_events))
    print("Event yield is:", np.sum(filtered_weights))
    filtered_events['Event.Weight'] = filtered_weights

    return filtered_events

def electron_vector(filtered_events):
    electrons = vector.zip({
        'pt': filtered_events['Electron.PT'],
        'eta': filtered_events['Electron.Eta'],
        'phi': filtered_events['Electron.Phi'],
        'mass': m_e})
    electrons = ak.drop_none(electrons, highlevel=True)
    
    return electrons

def muon_vector(filtered_events):
    muons = vector.zip({
        'pt': filtered_events['Muon.PT'],
        'eta': filtered_events['Muon.Eta'],
        'phi': filtered_events['Muon.Phi'],
        'mass': m_mu})
    muons = ak.drop_none(muons, highlevel=True)
    
    return muons

def NP_normalisation(events, k_factor):
    luminosity = 139 # fb^-1
    normalised_weight = (1 / len(events)) * luminosity * k_factor

    return normalised_weight

def export_to_feather(filtered_events):
    df = ak.to_dataframe(filtered_events, how="outer")
    df = df.reset_index()
    feather.write_feather(df, "WWW_data.feather", compression="lz4")
    
def __main__():
    plus_events_1 = import_raw_data()
    #plus_cross_section = 2.486
    #minus_cross_section = 1.096
    plus_k_factor = 136 / 73.69
    #minus_k_factor = 76 / 39.26

    plus_filtered_events_1, filtered_weights_1 = electron_muon_cuts(plus_events_1)
    
    plus_final_filtered_events_1 = jet_cuts(plus_filtered_events_1, plus_events_1, filtered_weights_1, plus_k_factor)
    
    export_to_feather(plus_final_filtered_events_1)

if __name__ == '__main__':
    __main__()

Number of events passing the selection cuts is: 3601
Event yield is: 0.00028192552
